In [26]:
import os
import pandas as pd

school_data_file = os.path.join("Resources", "schools_complete.csv")
students_data_file = os.path.join("Resources", "students_complete.csv")

In [27]:
students_df = pd.read_csv(students_data_file)
school_df = pd.read_csv(school_data_file)
students_df.tail()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score
39165,39165,Donna Howard,F,12th,Thomas High School,99,90
39166,39166,Dawn Bell,F,10th,Thomas High School,95,70
39167,39167,Rebecca Tanner,F,9th,Thomas High School,73,84
39168,39168,Desiree Kidd,F,10th,Thomas High School,99,90
39169,39169,Carolyn Jackson,F,11th,Thomas High School,95,75


In [98]:
# find the names that need fixed
students_names = students_df['student_name'].tolist()
students_to_fix = list()
prefixes = list()
suffixes = list()
for name in students_names:
    if len(name.split(" ")) >=3:    # find and save the name that potentially needs fixed
        students_to_fix.append(name)  
        if len(name.split(' ')[0]) <= 4:  # save the prefixes
            prefixes.append(name.split(' ')[0]) 
        else:
            pass
        if len(name.split(' ')[-1]) <= 3:    # save the suffixes
            suffixes.append(name.split(' ')[-1])

# make the prefixes and suffixes lists contain only unique
prefixes = list(set(prefixes)) 
suffixes = list(set(suffixes))
# print(suffixes)  # for Debugging and finding the unwanted prefixes and suffixes amoung the list of names that were saved by the filter.
suffixes_to_fix = [' MD', ' PhD', ' DDS', ' DVM']
prefixes_to_fix = ['Dr. ', 'Miss ', 'Mr. ', 'Mrs. ', 'Ms. ']
# print(students_to_fix)

In [99]:
# find and fix the prefixes
for pref in prefixes_to_fix:
    for index, name in enumerate(students_to_fix):
        if pref in (name[:len(pref)]):
            name = name[len(pref):]
            students_to_fix[index] = name

        else: 
            pass


# find and fix the suffixes
for suff in suffixes_to_fix:
    for index, name in enumerate(students_to_fix):
        if suff in (name[-(len(pref)):]):
            name = name[:-(len(suff))]
            students_to_fix[index] = name
               
        else:
            pass

print(students_to_fix)

['Richard Scott', 'Jordan Carson', 'Madeline Snyder', 'Dylan Taylor', 'Scott Gill', 'Madison Everett', 'Virginia Ramirez', 'Joseph Morales III', 'Angela Perkins', 'Heather Allen', 'Luke Lowery', 'Larry Hines', 'Emily Cardenas', 'Elizabeth Espinoza', 'Lisa Becker', 'Travis Young', 'Paula Fernandez', 'Xavier Bell II', 'Andrew Hanson', 'Roberto Wright', 'Dale Miller', 'Mary Gonzalez', 'Cindy Brown', 'James White', 'Donald Gill', 'Catherine Williams', 'Ashley Caldwell', 'Jared Campbell', 'Michael Moreno', 'Tara Parker', 'Shawn Williams', 'Alicia Martinez', 'Timothy Anderson', 'Paula Lee', 'Elijah Hall', 'Raymond Stone', 'John Dickerson', 'Alex Franklin', 'Jessica Dunn', 'Michael Stevenson', 'Jason Jones', 'Adriana Wade', 'Erica Hurley', 'Garrett Wright', 'James Allen', 'Timothy Harper', 'Dale Brown', 'Christina Moore', 'Angela Peterson', 'Jennifer Ferguson', 'Anthony Wilson', 'Jessica Lopez', 'David Miller Jr.', 'John Jones', 'Samantha Brown', 'Cynthia Hansen', 'Rebecca Montoya', 'Jacob Ma